# Imports

In [ ]:
import importlib.resources as resources
import numpy as np
import pydot

from anypick_dk.constants import (
    IIWA_LEN, q_BotShelfPlace, q_MidShelfPlace, q_Object, q_TopShelfPlace,
    WSG_CLOSED, WSG_LEN, WSG_OPENED, WSG_VEL_BOUND
)
from anypick_dk.planner import Planner
from anypick_dk.sim_environment import SimEnvironment
from anypick_dk.utils import concat_iiwa_traj, concat_wsg_traj, create_wsg_traj
from IPython.display import SVG, display
from pydrake.all import GcsTrajectoryOptimization, HPolyhedron, LoadIrisRegionsYamlFile, Point
Subgraph = GcsTrajectoryOptimization.Subgraph

In [ ]:
scenario_file = str(resources.files("anypick_dk") / "scenarios" / "shelf_with_objects.yaml")
sim_env = SimEnvironment(scenario_file)
planner = Planner(sim_env)
sim_env.publish_diagram()